<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Marfida%20Alexander/A_Marfida_stajirovka_profpatolog_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стажировка. Профпатолог (INSPECTRUM CLINIC)
- Анализ и обработка исходных данных

## Загрузка исходных данных

In [ ]:
# Подключение библиотек

# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Создание статистических графиков
import seaborn as sns

# Табличные данные
from fastai.tabular import *

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
##
# gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l9/wine_base.csv', None, quiet=True)
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx', None, quiet=True)


'%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'

Загруженный excel переименован в ICdata.xlsx

In [ ]:
# Монтирование Goodle-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Чтение исходного excel в переменную
# df_backup = pd.read_excel('ICdata.xlsx')
df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata.xlsx')

In [ ]:
# Запись или чтение последней версии обработанных исходных данных
# df.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemp.xlsx', , index=False)
#df_backup.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemp.xlsx', index=False) #!#, encoding='utf-8-sig')
    #!# /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
    #!# return func(*args, **kwargs)
df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICtemt.xlsx')

In [ ]:
# Создание рабочей копии датафрейма
df = df_backup.copy() # (deep=True) - default

## Служебные функции

### Функция просмотра содержания ненулевых уникальных значений полей в выбранных колонках

In [ ]:
def prn_unique_txt_col(df, *arg, sort_asc = False):
  for colname in arg:
    column_list = list(df.loc[~df[colname].isnull(), colname].unique())
    count_list = len(column_list)
    print(f'*** {colname}({count_list} уникальных): ***')
    to_join = sorted(column_list) if sort_asc else column_list
    print('\n'.join(to_join))
    print()

### Функция подсчёта заполненности колонок

In [ ]:
# Аргументы - параметры
def count_columns_fill(df, *arg):
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg:
    print(f'*** {colname}: ***') 
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')

In [ ]:
# Аргументы - список
def count_columns_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for colname in arg_list:
    count_list += 1
    print(f'*** {colname}: ***') 
    print('Количество непустых элементов: ', len(df[~df[colname].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[colname].isnull(), colname].unique()), '\n')
  print('Количество столбцов: ', count_list)

### Функция удаления из списка малоинформативных столбцов

In [ ]:
def reduce_colomns_unique_1(df, dfcols_to_save):
  ret = []
  print('Входящий размер списка столбцов: ', len(dfcols_to_save))
  for colname in dfcols_to_save:
    if len(df.loc[~df[colname].isnull(), colname].unique()) != 1:
      ret.append(colname)
  print('Исходящий размер списка столбцов: ', len(ret))
  ret
  return ret

### Функция сравнения пары списков на совпадение

In [ ]:
def is_equal_lst_cols(col1, col2):
  res = [x for x in col1 + col2 if x not in col1 or x not in col2] 
  if not res: 
    print("Списки совпаают")
  else: 
    print("*** Несовпадающие элементы списков: ***") 
    print('\n'.join(res)) 

### Функция сравнения пары колонок датафрейма на совпадение

In [ ]:
def is_equal_df_cols(df, col1, col2):
  dfcmp = np.where(df[col1] == df[col2], True, False)
  df_not_equal_indexes = df[~dfcmp].index
  print(df[~dfcmp][[col1, col2]])
#  return df_not_equal_indexes

## Исследование и преобразование исходного набора данных

### Обзор структуры и содержания данных

In [ ]:
# Размерность исходная
df.shape

(1982, 4587)

In [ ]:
# Характеристика по типам данных и их количеству
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


#### Удаление дубликатов строк и солбцов

In [ ]:
# Первые из строк, дублирующиеся далее в датафрейме
df[df.duplicated()]

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
392,2997d782-55ab-11ed-8108-0cc47aab8067,116fe6a5-455f-11ed-8107-0cc47aab8067,1977-03-06,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,61d271b9-55ab-11ed-8108-0cc47aab8067,c547414b-732c-11e6-8360-b8975a94e496,1963-01-17,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,e334cb0e-55a6-11ed-8108-0cc47aab8067,116fe6a8-455f-11ed-8107-0cc47aab8067,1971-02-22,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,549479ee-55aa-11ed-8108-0cc47aab8067,24086b2f-4bf7-11e7-80d9-708bcd7f83cd,1976-06-16,Женский,Женский,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,832c79bc-55b5-11ed-8108-0cc47aab8067,39f65c6b-34f7-11e9-80ce-0cc47aab8067,1982-03-19,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,fb21176c-55bf-11ed-8108-0cc47aab8067,5ba8b3e2-81bf-11e9-80cf-0cc47aab8067,1969-09-02,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,30b084e0-5678-11ed-8108-0cc47aab8067,9fc8140c-bd93-11e8-80ce-0cc47aab8067,1970-01-21,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,00d62b6d-5678-11ed-8108-0cc47aab8067,dae940bb-c84c-11e8-80ce-0cc47aab8067,1970-09-06,Женский,Женский,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,723358f4-568f-11ed-8108-0cc47aab8067,5aafffb4-c344-11e9-80cf-0cc47aab8067,1979-03-04,Мужской,Мужской,2022-10-28,Периодический,Периодический,False,963d16f4-d69e-11e7-80de-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df.duplicated()]['ЗаключениеМК']

392                      Годен
395                      Годен
396            ВременноНегоден
398                      Годен
399                      Годен
410                      Годен
411                      Годен
429                      Годен
430                      Годен
431                      Годен
432                      Годен
433                      Годен
435                      Годен
436                      Годен
439                      Годен
448                      Годен
449                      Годен
450                      Годен
451                      Годен
452     ГоденСКоррекциейЗрения
470     ГоденСКоррекциейЗрения
471                      Годен
496                      Годен
506                      Годен
507                      Годен
509                      Годен
510                      Годен
585     ГоденСКоррекциейЗрения
586                      Годен
681                      Годен
1085                     Годен
1086                     Годен
1087    

In [ ]:
# Количество повторяющихся строк в DataFrame
len(df) - len(df.drop_duplicates())

56

In [ ]:
# Удаление дубликатов строк
df.drop_duplicates(inplace=True)

In [ ]:
# Размер данных после удаления дубликатов строк
df.shape

(1926, 4587)

In [ ]:
1982 - 1926

56

Удалено 56 дубликатов строк

#### Проверка наличия пустых столбцов и их удаление:

In [ ]:
dn = df.isnull().sum()
print('Количество пустых столбцов:', len(dn[dn == 1926]))
df.dropna(axis=1, how='all', inplace=True)
df.shape

Количество пустых столбцов: 884


(1926, 3703)

- Далее работаем с датафреймом:
      (1926, 3703)

In [ ]:
df.info() #(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1926 entries, 0 to 1981
Columns: 3703 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(369), object(3290)
memory usage: 54.4+ MB


#### Анализ заполненности столбцов

In [ ]:
df.count()

ДокументПрохождениеМедосмотра                                             1926
Клиент                                                                    1926
КлиентДатаРождения                                                        1926
КлиентПол                                                                 1926
КлиентПолПредставление                                                    1926
                                                                          ... 
Стоматология2_ВП2_ЗначениеПредставление                                      1
Терапия2_ЗаключениеФОГК_Значение                                             1
Терапия2_ЗаключениеФОГК_ЗначениеПредставление                                1
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение                    1
Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление       1
Length: 3703, dtype: int64

##### Самые заполненные столбцы, если они не информационно-учётного характера, то могут быть существенными для обучения НС

- Полностью заполненные столбцы

In [ ]:
list(df.count().index[df.count().values == 1926])

['ДокументПрохождениеМедосмотра',
 'Клиент',
 'КлиентДатаРождения',
 'КлиентПол',
 'КлиентПолПредставление',
 'ДатаЗавершенияМедосмотра',
 'ТипМедосмотра',
 'ТипМедосмотраПредставление',
 'ПсихОсвидетельствование',
 'Контрагент',
 'Профессия',
 'ВредныеФакторы']

In [ ]:
df[['КлиентПол', 'КлиентПолПредставление']]

,КлиентПол,КлиентПолПредставление
0,Женский,Женский
1,Мужской,Мужской
2,Мужской,Мужской
3,Мужской,Мужской
4,Мужской,Мужской
...,...,...
1977,Мужской,Мужской
1978,Мужской,Мужской
1979,Мужской,Мужской
1980,Женский,Женский


In [ ]:
# Проверка совпадающих колонок
is_equal_df_cols(df, 'КлиентПол', 'КлиентПолПредставление')

Empty DataFrame
Columns: [КлиентПол, КлиентПолПредставление]
Index: []


различий нет - полное совпадение

In [ ]:
# Проверка уникальности в колонках
prn_unique_txt_col(df, 'КлиентПол')

*** КлиентПол(2 уникальных): ***
Женский
Мужской



Что за ↓"Контрагенты"↓?  (177 уникальных!)



In [ ]:
prn_unique_txt_col(df, 'Контрагент')

*** Контрагент(177 уникальных): ***
23c173a7-30d9-11e7-80d8-708bcd7f83cd
a6d559b0-7835-11eb-80e8-0cc47aab8067
a1e92db1-2ea1-11e9-80ce-0cc47aab8067
1ef9e604-5432-11ec-80fc-0cc47aab8067
1d072795-48e4-11e6-82d8-d897ba0a5c60
992c8208-4e0d-11ed-8107-0cc47aab8067
e8b70525-bcef-11e8-80ce-0cc47aab8067
a495b8a2-fe8a-11ec-8103-0cc47aab8067
16552adb-a1a4-11eb-80f0-0cc47aab8067
e935abf8-4ed4-11e9-80ce-0cc47aab8067
4c79459c-b702-11e8-80ce-0cc47aab8067
ac198fbb-2568-11ea-80d7-0cc47aab8067
8cd212c1-84c9-11ec-80fc-0cc47aab8067
caa5478c-7a99-11ec-80fc-0cc47aab8067
e71ae138-fd17-11ea-80e2-0cc47aab8067
4d0f5a2d-1589-11ea-80d3-0cc47aab8067
ff5c0c64-6975-11e8-80ce-0cc47aab8067
9fe38364-b4b5-11ec-8100-0cc47aab8067
f6b57b5c-836d-11e8-80ce-0cc47aab8067
295d0691-22da-11ed-8104-0cc47aab8067
b1640169-de75-11e9-80d1-0cc47aab8067
31f0b8ac-b66a-11e8-80ce-0cc47aab8067
0ca6b915-6291-11e9-80ce-0cc47aab8067
9fd0ecba-afea-11e7-80dc-708bcd7f83cd
150475e5-8431-11e8-80ce-0cc47aab8067
226f6094-63db-11e8-80ce-0cc47aab8067
5a

In [ ]:
# Проверка заполненности столбца(-ов)
count_columns_fill(df, ['Контрагент', 'Профессия', 'ТипМедосмотраПредставление'])

Текущая структура датафрейма:  (1926, 3703) 

*** Контрагент: ***
Количество непустых элементов:  1926
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1926
Количество уникальных элементов:  690 

*** ТипМедосмотраПредставление: ***
Количество непустых элементов:  1926
Количество уникальных элементов:  2 



In [ ]:
prn_unique_txt_col(df, 'ТипМедосмотраПредставление')

*** ТипМедосмотраПредставление(2 уникальных): ***
Периодический
Предварительный



In [ ]:
prn_unique_txt_col(df, 'Профессия', sort_asc = True)

*** Профессия(690 уникальных): ***
Авиамеханик
Авиамеханик 
Авиамеханик Авиамеханик
Авиатехник  ПиД
Авиатехник АиРЭО
Авиатехник ПиД
Агент-комплектовщик
Агроном 1 категории
Администратор
Администратор-кассир
Анестезиолог
Ассистент врача-стоматолога
Бармен
Барменеджер
Бригадир
Бригадир-электромонтажник по обслуживанию и ремонту электрооборудования
Бригадир-электромонтер по обслуживанию и ремонту электрооборудования
Бульдозерист
Бурильщик
Бурильщик капитального ремонта скважин
Бухгалтер
Бухгалтер 1 категории
Вальщик леса
Вальщик леса 6 разряда
Вальщик леса 6 разряда 
Ведущий агроном
Ведущий агроном по защите растений
Ведущий агроном по семеноводству
Ведущий бухгалтер
Ведущий геодезист
Ведущий геофизик
Ведущий инженер
Ведущий инженер 
Ведущий инженер (топ оборудование)
Ведущий инженер - проектировщик
Ведущий инженер АиРЭО
Ведущий инженер ПиД
Ведущий инженер по бурению 
Ведущий инженер по радиационной безопасности (класс вредности 3.1)
Ведущий инженер по сварке
Ведущий инженер-дефектоскопис

In [ ]:
# Исправление ошибок
error = {
    'Ведущий специалист по организации обуения':'Ведущий специалист по организации обучения',
    'Водитель по доставке готовй продукции':'Водитель по доставке готовой продукции',
    'Авиамеханик Авиамеханик':'Авиамеханик',
    'Водитель автомобиля 1 клаасса 5 разряда':'Водитель автомобиля 1 класса 5 разряда',
    'Водитель автомобиля- грузчик':'Водитель автомобиля-грузчик',
    'Водитель транспортного средства категории С':'Водитель транспортного средства категории "С"',
    'Водитель вездехода 6 разряд':'Водитель вездехода 6 разряда',
    'Ведущий инженер-электроник ( 5 класса)':'Ведущий инженер-электроник (5 класса)',
    'Взрывник 6 раазряда':'Взрывник 6 разряда',
    'Взрывник 6 го разряда':'Взрывник 6 разряда',
    'Водитель автомобиля (камаз-манипулятор, ломовоз':'Водитель автомобиля (камаз-манипулятор, ломовоз)',
    'Водитель автомобиля- грузчик':'Водитель автомобиля-грузчик',
    'Врач - стоматолог терапевт':'Врач стоматолог-терапевт',
    'Врач стоматолог- общей практики':'Врач стоматолог-общей практики',
    'Врач стоматолог - ортопед':'Врач стоматолог-ортодонт',
    'Врач стоматолог – детский, Врач - ортодонт':'Врач стоматолог-детский',
    'Врач-стоматолог':'Врач стоматолог',
    'Врач-стоматолог-ортопед':'Врач стоматолог-ортопед',
    'Врач-стоматолог-терапевт':'Врач стоматолог-терапевт',
    'Врач-стоматолог-хирург':'Врач стоматолог-хирург',
    'Грузчик — комплектовщик':'Грузчик-комплектовщик',
    'Дефектоскопист ренгено-гаммаграфирования НК':'Дефектоскопист ренnгено-,гаммаграфирования НК',
    'Дефектоскопист рентгено-гаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгеногаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгенограммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгеногаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист рентгенограммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Дефектоскопист ренnгено-,гаммаграфирования НК':'Дефектоскопист рентгено-,гаммаграфирования НК',
    'Инженер по буровым работам (ответственный  руководитель буровых работ)':'Инженер по буровым работам (ответственный руководитель буровых работ)',
    'Инженер по взрывным работам(ответственный руководитель взрывных работ) 1 категории':'Инженер по взрывным работам (ответственный руководитель взрывных работ) 1 категории',
    'Зав складом':'Зав. складом',
    'Зав. Склад':'Зав. cкладом'
}
df['Профессия'] = df['Профессия'].replace(error)

Полностью столбец "Профессия" не отредактирован. Доделать на новом датафрейме

In [ ]:
# Удаление концевых пробелов
df['Профессия'] = df['Профессия'].str.strip()

In [ ]:
prn_unique_txt_col(df, 'Профессия', sort_asc = True)

*** Профессия(625 уникальных): ***
Авиамеханик
Авиатехник  ПиД
Авиатехник АиРЭО
Авиатехник ПиД
Агент-комплектовщик
Агроном 1 категории
Администратор
Администратор-кассир
Анестезиолог
Ассистент врача-стоматолога
Бармен
Барменеджер
Бригадир
Бригадир-электромонтажник по обслуживанию и ремонту электрооборудования
Бригадир-электромонтер по обслуживанию и ремонту электрооборудования
Бульдозерист
Бурильщик
Бурильщик капитального ремонта скважин
Бухгалтер
Бухгалтер 1 категории
Вальщик леса
Вальщик леса 6 разряда
Ведущий агроном
Ведущий агроном по защите растений
Ведущий агроном по семеноводству
Ведущий бухгалтер
Ведущий геодезист
Ведущий геофизик
Ведущий инженер
Ведущий инженер (топ оборудование)
Ведущий инженер - проектировщик
Ведущий инженер АиРЭО
Ведущий инженер ПиД
Ведущий инженер по бурению
Ведущий инженер по радиационной безопасности (класс вредности 3.1)
Ведущий инженер по сварке
Ведущий инженер-дефектоскопист
Ведущий инженер-конструктор (5 класс)
Ведущий инженер-электроник (1 класса) (

In [ ]:
prn_unique_txt_col(df, 'ВредныеФакторы')

*** ВредныеФакторы(515 уникальных): ***
11.4
4.2.5
11.1
4.3.1,10,11.1
1.50,4.2.5,4.1,1.8.1.1
1.30.1,1.34.1,1.37.1
5.1,6.1,6.2
5.1,6
5.1,6.2,6.1
5.1
6.1,6.2,15
6.1,6.2
1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.34.1,1.37.1,1.50
1.34.1,1.19.1,1.14.2,1.50,1.37.1,1.8.1.1,1.8.2.1
1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.37.1,1.50,1.34.1
1.8.1.1,1.8.2.1,1.14.2,1.34.1,1.37.1,1.50,1.19.1
23
4.4
1.23,1.46,3.4
1.50,1.34.1,1.37.1
1.8.1.1,1.50
1.34.1,1.37.1,1.50
1.50,4.3.1
1.50,1.8.1.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1.1,1.34.1,1.50
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.37.1,1.50
1.37.1,1.29.2,1.50,1.14.2,1.8.1.1,17,6.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.50,1.37.1
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1,1.34.1,1.50
1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.37.1,1.50,6.1
1.8.1.1,1.14.2,1.29.2,1.37.1,1.50,6.1,17
1.14.2,1.29.2,1.29.4

In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', 'ЗаключениеМКПредставление', 'Заключение')

*** ЗаключениеМК(9 уникальных): ***
Годен
ГоденСКоррекциейЗрения
ГоденБезРаботНаВысотах
НуждаетсяВДообследованииИЛечении
ВременноНегоден
ОграниченноГоден
машинист бульдозера
Газоспасатель
 Газоспасатель

*** ЗаключениеМКПредставление(9 уникальных): ***
Годен к работе в указанных условиях
Годен к работе в указанных условиях (с коррекцией зрения)
Годен к работе в указанных условиях (без работ на высотах)
Нуждается в дообследовании и лечении
Временно не годен
Ограниченно годен
тракторист
ГоденСКоррекциейЗрения
Годен

*** Заключение(14 уникальных): ***
Годен к работе в указанных условиях
Годен к работе в указанных условиях (с коррекцией зрения)
Годен к работе в указанных условиях (с коррекцией зрения), кроме п.6.1
Годен к работе в указанных условиях (без работ на высотах) Противопоказана работа связанная с  высотой ,подьем и перемещение тяжестей ,длительной ходьбой.
Нуждается в дообследовании и лечении
Временно не годен
Ограниченно годен
Годен к работе в указанных условиях (с коррекцией зр

In [ ]:
# is_equal_df_cols(df, 'ЗаключениеМК', 'Заключение') #'ЗаключениеМКПредставление')
is_equal_df_cols(df, 'ЗаключениеМК', 'ЗаключениеМКПредставление')

                ЗаключениеМК  \
0                      Годен   
1                      Годен   
2     ГоденСКоррекциейЗрения   
3                      Годен   
4     ГоденСКоррекциейЗрения   
...                      ...   
1977                     NaN   
1978                     NaN   
1979                     NaN   
1980                     NaN   
1981                   Годен   

                              ЗаключениеМКПредставление  
0                   Годен к работе в указанных условиях  
1                   Годен к работе в указанных условиях  
2     Годен к работе в указанных условиях (с коррекц...  
3                   Годен к работе в указанных условиях  
4     Годен к работе в указанных условиях (с коррекц...  
...                                                 ...  
1977                                                NaN  
1978                                                NaN  
1979                                                NaN  
1980                               

In [ ]:
#
count_columns_fill(df, ['ЗаключениеМК', 'ЗаключениеМКПредставление', 'Заключение'])

Текущая структура датафрейма:  (1926, 3703) 

*** ЗаключениеМК: ***
Количество непустых элементов:  1519
Количество уникальных элементов:  9 

*** ЗаключениеМКПредставление: ***
Количество непустых элементов:  1519
Количество уникальных элементов:  9 

*** Заключение: ***
Количество непустых элементов:  1507
Количество уникальных элементов:  14 



##### Починим целевой столбец 

In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', sort_asc = True)

*** ЗаключениеМК(9 уникальных): ***
 Газоспасатель
ВременноНегоден
Газоспасатель
Годен
ГоденБезРаботНаВысотах
ГоденСКоррекциейЗрения
НуждаетсяВДообследованииИЛечении
ОграниченноГоден
машинист бульдозера



In [ ]:
# Удаление строк с ошибками
values = [' Газоспасатель', 'Газоспасатель', 'машинист бульдозера']
print(df.shape)
df = df[df['ЗаключениеМК'].isin (values) == False]
print(df.shape)

(1926, 3703)
(1922, 3703)


In [ ]:
prn_unique_txt_col(df, 'ЗаключениеМК', sort_asc = True)

*** ЗаключениеМК(6 уникальных): ***
ВременноНегоден
Годен
ГоденБезРаботНаВысотах
ГоденСКоррекциейЗрения
НуждаетсяВДообследованииИЛечении
ОграниченноГоден



Приводим целевой столбец с трафарету: 'Годен', 'Не годен', 'Ограниченно годен'

In [ ]:
count_columns_fill(df, ['ЗаключениеМК'])

Текущая структура датафрейма:  (1922, 3703) 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  6 



In [ ]:
change = {
    'ВременноНегоден':'Не годен',
    'ГоденБезРаботНаВысотах':'Годен',
    'ГоденСКоррекциейЗрения':'Годен',
    'НуждаетсяВДообследованииИЛечении':'Не годен',
    'ОграниченноГоден':'Ограниченно годен'
}
df['ЗаключениеМК'] = df['ЗаключениеМК'].replace(change)

In [ ]:
count_columns_fill(df, 'Хирургия2_МКБ101', 'Гинекология2_НеИзмененоСОсобенностями_Значение', 'Гинекология2_НеИзмененоСОсобенностями_ЗначениеПредставление')

Текущая структура датафрейма:  (1922, 3703) 

*** Хирургия2_МКБ101: ***
Количество непустых элементов:  78
Количество уникальных элементов:  2 

*** Гинекология2_НеИзмененоСОсобенностями_Значение: ***
Количество непустых элементов:  23
Количество уникальных элементов:  1 

*** Гинекология2_НеИзмененоСОсобенностями_ЗначениеПредставление: ***
Количество непустых элементов:  23
Количество уникальных элементов:  1 



In [ ]:
# Выборка столбцов с заданной заполненностью 
not_empty_vals_in_column = 1000
col_fill = list(df.count().index[df.count().values >= not_empty_vals_in_column])
print('Размерность датафрейма: ', df.shape)
print(f'Количество столбцов, удовлетворяющих условию "{not_empty_vals_in_column}" равно: ', len(col_fill))

Размерность датафрейма:  (1926, 3703)
Количество колонок, удовлетворяющих условию "1000" равно:  426


In [ ]:
len(col_fill)

426

### Анализ столбцов типа 'float64'

In [ ]:
# Все цифровые колонки
df.select_dtypes(include=['float64']).columns

Index(['ПсихиатрияНаркология1_Профессиональное1',
       'ПсихиатрияНаркология1_Чистая_Значение',
       'ПсихиатрияНаркология1_СоСледамиВВИнъекций_Значение',
       'ПсихиатрияНаркология1_Контактен_Значение',
       'ПсихиатрияНаркология1_НеКонтактен_Значение',
       'ПсихиатрияНаркология1_Адекватен_Значение',
       'ПсихиатрияНаркология1_НеАдекватен_Значение',
       'Офтальмология1_Профессиональное1',
       'Офтальмология1_ДискиЗрительныхНервовБледноРозовые_Значение',
       'Офтальмология1_ГраницыЧеткие_Значение',
       ...
       'ТерапевтНеИсп1_ТемператураТелаТерапевт_ЗначениеПредставление',
       'ТерапевтНеИсп1_ИМТ_Значение',
       'ТерапевтНеИсп1_АртериальноеДавление21_Значение',
       'ТерапевтНеИсп1_АртериальноеДавление21_ЗначениеПредставление',
       'ТерапевтНеИсп1_Сатурация_Значение',
       'ТерапевтНеИсп1_Сатурация_ЗначениеПредставление',
       'Терапия3_ОбщийХолестерин_Значение',
       'Терапия4_ОбщийХолестерин_Значение', 'Гинекология2_Выкидышей_Значение',
  

In [ ]:
count_columns_fill(df, ['ТерапевтНеИсп1_АртериальноеДавление21_Значение', 'ТерапевтНеИсп1_АртериальноеДавление21_ЗначениеПредставление'])

Текущая структура датафрейма:  (1922, 3703) 

*** ТерапевтНеИсп1_АртериальноеДавление21_Значение: ***
Количество непустых элементов:  8
Количество уникальных элементов:  2 

*** ТерапевтНеИсп1_АртериальноеДавление21_ЗначениеПредставление: ***
Количество непустых элементов:  8
Количество уникальных элементов:  2 

Количество столбцов:  2


In [ ]:
df['ТерапевтНеИсп1_АртериальноеДавление21_Значение']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1977   NaN
1978   NaN
1979   NaN
1980   NaN
1981   NaN
Name: ТерапевтНеИсп1_АртериальноеДавление21_Значение, Length: 1926, dtype: float64

- Гипотеза: 
  - Числовые столбцы малоинформативны. 
- Решение:
  - Исключаются их первого подхода к обучению

### Сокращение датафрейма

In [ ]:
# Формирование списка сохраняемых столбцов
# 
dfcols_to_save = ['КлиентДатаРождения', 'КлиентПол', 'Контрагент', 
                  'Профессия', 'ЗаключениеМК', 'ВредныеФакторы']

dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Обследование')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Заключение_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ГоденНегоден_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('Диагноз_ЗначениеПредставление')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ПрофнепригодностьВременнаяПредставление1')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('ПрофнепригодностьВременнаяПредставление2')].to_numpy())
dfcols_to_save = dfcols_to_save + list(df.columns[df.columns.str.contains('МКБ101')].to_numpy())

dfcols_to_save

['КлиентДатаРождения',
 'КлиентПол',
 'Контрагент',
 'Профессия',
 'ЗаключениеМК',
 'ВредныеФакторы',
 'ФункциональнаяДиагностика1_ОбследованиеПроведено_Значение',
 'ФункциональнаяДиагностика1_ОбследованиеПроведено_ЗначениеПредставление',
 'Дерматовенерология1_ОбследованиеНаПедикулез_Значение',
 'Дерматовенерология1_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'ЭКГНеИсп1_ОбследованиеПроведено_Значение',
 'ЭКГНеИсп1_ОбследованиеПроведено_ЗначениеПредставление',
 'Хирургия1_ОбследованиеНаПедикулез_Значение',
 'Хирургия1_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'ФункциональнаяДиагностика2_ОбследованиеПроведено_Значение',
 'ФункциональнаяДиагностика2_ОбследованиеПроведено_ЗначениеПредставление',
 'Дерматовенерология2_ОбследованиеНаПедикулез_Значение',
 'Дерматовенерология2_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'Хирургия3_ОбследованиеНаПедикулез_Значение',
 'Хирургия3_ОбследованиеНаПедикулез_ЗначениеПредставление',
 'Хирургия2_ОбследованиеНаПедикулез_Значение',
 'Хиру

In [ ]:
# Анализ заполненности выбранных столбцов
count_columns_fill(df, dfcols_to_save)

Текущая структура датафрейма:  (1922, 3703) 

*** КлиентДатаРождения: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  1794 

*** КлиентПол: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  2 

*** Контрагент: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  623 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  3 

*** ВредныеФакторы: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  512 

*** ФункциональнаяДиагностика1_ОбследованиеПроведено_Значение: ***
Количество непустых элементов:  430
Количество уникальных элементов:  1 

*** ФункциональнаяДиагностика1_ОбследованиеПроведено_ЗначениеПредставление: ***
Количество непустых элементов:  430
Количество уникальных элементов:  1 

*** Дерматовенерология1_ОбследованиеНаПедикулез_Значение: ***
Количе

Избавимся от столбцов, мало приходных для качества обучения: с единственным выбором (количество уникальных = 1)

In [ ]:
dfcols_to_save = reduce_colomns_unique_1(df, dfcols_to_save)

Входящий размер списка столбцов:  205
Исходящий размер списка столбцов:  78


In [ ]:
count_columns_fill(df, dfcols_to_save)

Текущая структура датафрейма:  (1922, 3703) 

*** КлиентДатаРождения: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  1794 

*** КлиентПол: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  2 

*** Контрагент: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  623 

*** ЗаключениеМК: ***
Количество непустых элементов:  1515
Количество уникальных элементов:  3 

*** ВредныеФакторы: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  512 

*** УЗИДиагностика1_Заключение_ЗначениеПредставление: ***
Количество непустых элементов:  403
Количество уникальных элементов:  177 

*** УЗИДиагностика2_Заключение_ЗначениеПредставление: ***
Количество непустых элементов:  16
Количество уникальных элементов:  10 

*** ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление: ***
Количество непустых элементов: 

In [ ]:
prn_unique_txt_col(df, 'Заключение', sort_asc = True)

*** Заключение(13 уникальных): ***
Временно не годен
Годен к работе в указанных условиях
Годен к работе в указанных условиях (без работ на высотах) Противопоказана работа связанная с  высотой ,подьем и перемещение тяжестей ,длительной ходьбой.
Годен к работе в указанных условиях (с коррекцией зрения)
Годен к работе в указанных условиях (с коррекцией зрения) кроме п.6.1, 6.2.
Годен к работе в указанных условиях (с коррекцией зрения) кроме п.6.1.
Годен к работе в указанных условиях (с коррекцией зрения), кроме п. 6 и п.14
Годен к работе в указанных условиях (с коррекцией зрения), кроме п. 6.1
Годен к работе в указанных условиях (с коррекцией зрения), кроме п.17
Годен к работе в указанных условиях (с коррекцией зрения), кроме п.6.1
Годен к работе в указанных условиях, кроме п 6.2
Нуждается в дообследовании и лечении
Ограниченно годен



In [ ]:
df[df['ЗаключениеМК'] == 'Ограниченно годен']

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
207,86cc0f04-4b86-11ed-8107-0cc47aab8067,5dbfe97f-0477-11eb-80e2-0cc47aab8067,1991-05-31,Мужской,Мужской,2022-10-14,Периодический,Периодический,False,e8b70525-bcef-11e8-80ce-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,db0b9b12-6b06-11ed-8109-0cc47aab8067,9aa707e6-f7e0-11ea-80e2-0cc47aab8067,1983-04-06,Мужской,Мужской,2022-11-01,Периодический,Периодический,False,f4590331-8ef1-11e7-80db-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df['ЗаключениеМК'] == 'Не годен']

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
177,2aeafece-4874-11ed-8107-0cc47aab8067,f48854a1-4868-11ed-8107-0cc47aab8067,1987-03-06,Мужской,Мужской,2022-10-10,Периодический,Периодический,False,10bd7eb8-3586-11ed-8105-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,eab3ac0a-484c-11ed-8107-0cc47aab8067,ddb457d3-1451-11e7-80d3-708bcd7f83cd,1986-07-23,Мужской,Мужской,2022-10-10,Периодический,Периодический,False,e8b70525-bcef-11e8-80ce-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,32503214-55ac-11ed-8108-0cc47aab8067,116fe6a1-455f-11ed-8107-0cc47aab8067,1961-08-12,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,fc62264f-55aa-11ed-8108-0cc47aab8067,116fe6a3-455f-11ed-8107-0cc47aab8067,1968-07-22,Мужской,Мужской,2022-10-27,Периодический,Периодический,False,bd085484-1d16-11ed-8103-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479,86c6a4ea-58c9-11ed-8108-0cc47aab8067,70a2abb0-045d-11ea-80d3-0cc47aab8067,1969-07-14,Женский,Женский,2022-10-31,Периодический,Периодический,False,f92e1e40-9f09-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
978,fc32d76f-60bc-11ed-8109-0cc47aab8067,325d51cc-60a0-11ed-8109-0cc47aab8067,1987-10-26,Мужской,Мужской,2022-11-10,Периодический,Периодический,False,c9d539ae-91a3-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1228,ccc98b32-6566-11ed-8109-0cc47aab8067,f01db06a-5a81-11ed-8108-0cc47aab8067,1966-08-20,Мужской,Мужской,2022-11-16,Периодический,Периодический,False,3357cf50-775a-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1752,126d913c-6c74-11ed-8109-0cc47aab8067,0e6eb1ea-665f-11ed-8109-0cc47aab8067,1968-01-01,Мужской,Мужской,2022-11-25,Периодический,Периодический,False,bb4d4c59-1f29-11eb-80e3-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1764,cfb68f13-6ecf-11ed-8109-0cc47aab8067,baaae8f7-6210-11ec-80fc-0cc47aab8067,1970-01-01,Мужской,Мужской,2022-11-28,Периодический,Периодический,False,0b15bc61-618c-11ec-80fc-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949,e4512186-7062-11ed-8109-0cc47aab8067,9796d136-084d-11eb-80e3-0cc47aab8067,1963-08-12,Мужской,Мужской,2022-11-30,Периодический,Периодический,False,36b24009-f108-11ea-80e2-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,Рекомендации: лекарственные препараты принимат...,Рекомендации: лекарственные препараты принимат...,NaN,NaN


### Удаляем малоинформативные данные, 
### получая датафрейм для подготовки датасета обучения:

In [ ]:
# Удаление из датафрейма всех не выбранных для сохранения колонок
i = 0
try:
  while True:
    for col_to_save in dfcols_to_save:
      if df.columns[i] == col_to_save:
        i += 1
        continue
    else:
      df.drop(df.columns[i], axis= 1 , inplace=True)
except IndexError:
  df.shape

In [ ]:
df.shape

(1922, 78)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1922 entries, 0 to 1981
Data columns (total 78 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   КлиентДатаРождения                                              1922 non-null   datetime64[ns]
 1   КлиентПол                                                       1922 non-null   object        
 2   Контрагент                                                      1922 non-null   object        
 3   Профессия                                                       1922 non-null   object        
 4   ЗаключениеМК                                                    1515 non-null   object        
 5   ВредныеФакторы                                                  1922 non-null   object        
 6   ПсихиатрияНаркология1_МКБ101                                    1468 non-null   object  

Удалим самые неинформационные столбцы с заполненностью < 10

In [ ]:
df.drop('Терапия3_МКБ101', axis= 1 , inplace=True)
df.drop('Терапия3_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('Терапия4_МКБ101', axis= 1 , inplace=True)
df.drop('Терапия4_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('ТерапевтНеИсп1_Диагноз_ЗначениеПредставление', axis= 1 , inplace=True)
df.drop('ТерапевтНеИсп1_МКБ101', axis= 1 , inplace=True)
df.drop('Эндоскопия1_МКБ101', axis= 1 , inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1922 entries, 0 to 1981
Data columns (total 71 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   КлиентДатаРождения                                              1922 non-null   datetime64[ns]
 1   КлиентПол                                                       1922 non-null   object        
 2   Контрагент                                                      1922 non-null   object        
 3   Профессия                                                       1922 non-null   object        
 4   ЗаключениеМК                                                    1515 non-null   object        
 5   ВредныеФакторы                                                  1922 non-null   object        
 6   ПсихиатрияНаркология1_МКБ101                                    1468 non-null   object  

In [ ]:
df.shape

(1922, 71)

### Сохраняем очищенный датафрейм

In [ ]:
df.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71).xlsx', index=False)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 

---